## Import necessary libraries

In [1]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from keras.datasets import cifar10
from skimage.feature import local_binary_pattern

## Load CIFAR-10 dataset

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

## Convert images to grayscale and resize to (64, 64)

In [3]:
def preprocess(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_resized = cv2.resize(img_gray, (64, 64))
    return img_resized

X_train_processed = np.array([preprocess(img) for img in X_train])
X_test_processed = np.array([preprocess(img) for img in X_test])

# Extract LBP features from images
def extract_lbp_features(img):
    lbp = local_binary_pattern(img, 8, 1)
    hist, _ = np.histogram(lbp, bins=np.arange(0, 257), range=(0, 256))
    hist = hist.astype('float32')
    hist /= (hist.sum() + 1e-7)
    return hist

# Loop through each image in the training set
X_train_features = []
for img in X_train_processed:
    features = extract_lbp_features(img)
    X_train_features.append(features)

# Loop through each image in the test set
X_test_features = []
for img in X_test_processed:
    features = extract_lbp_features(img)
    X_test_features.append(features)

X_train_features = np.vstack(X_train_features)
X_test_features = np.vstack(X_test_features)

## Train SVM model

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train_features, y_train, test_size=0.2, random_state=42)


model = SVC(kernel='linear', C=1.0, random_state=42)
model.fit(X_train, y_train)

C:\Users\Safa Demirhan\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear', random_state=42)

## Evaluate model on test set

In [5]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.3366
